## Random Forest Classifier with Pipeline and Hyperparameter Tuning

In [1]:
import seaborn as sns

In [2]:
df=sns.load_dataset("tips")

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [6]:
df["time"]

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [7]:
from sklearn.preprocessing import LabelEncoder


In [8]:
encoder=LabelEncoder()

In [9]:
df["time"]=encoder.fit_transform(df["time"])

In [10]:
df["time"].unique()

array([0, 1])

In [11]:
x=df.drop(labels=['time'],axis=1)
y=df.time

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [14]:
x_train

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2
...,...,...,...,...,...,...
106,20.49,4.06,Male,Yes,Sat,2
14,14.83,3.02,Female,No,Sun,2
92,5.75,1.00,Female,Yes,Fri,2
179,34.63,3.55,Male,Yes,Sun,2


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  # TO handle missing values and used when we automate the process 
from sklearn.preprocessing import StandardScaler # Feature scaling
from sklearn.preprocessing import OneHotEncoder  # categorical to numercial
from sklearn.compose import ColumnTransformer

In [16]:
categorical_columns=["sex","smoker","day"]
numerical_column=["tip","total_bill","size"]


In [17]:
# FEATURE ENGINEERING AUTOMATION
num_pipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="median")), #missing values handled
        ("scaler",StandardScaler())   #feature scaling
    ]
)
categorical_pipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="most_frequent")), #missing values handled
        ("onehotencoder",OneHotEncoder()) # categorical to numerical
    ] 

)

In [18]:
preprocessor=ColumnTransformer(
    [
        ("num_pipeline",num_pipeline,numerical_column),
        ("categorical_pipeline",categorical_pipeline,categorical_columns)
    ]
)

In [19]:
x_train=preprocessor.fit_transform(x_train)
x_test=preprocessor.transform(x_test)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [21]:
# MODEL TRAINING AUTOMATION
models={
    
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier()
    
}

In [22]:
from sklearn.metrics import accuracy_score,r2_score

In [23]:
def evaluate_model(x_train,x_test,y_train,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(x_train,y_train)
        y_pred=model.predict(x_test)
        score=accuracy_score(y_test,y_pred)
        report[list(models.keys())[i]]=score
    return report
    

In [24]:
evaluate_model(x_train,x_test,y_train,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [25]:
classifier=RandomForestClassifier()

In [26]:
## Hyperparameter Tuning

In [27]:
params={
    "max_depth":[3,5,10,None],
    "n_estimators":[100,200,300],
    "criterion":["gini","entropy"]
}

In [28]:
from sklearn.model_selection import RandomizedSearchCV

In [29]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring="accuracy",cv=5,verbose=3)

In [30]:
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.974 total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.923 total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.974 total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.949 total time=   0.6s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=300;, score

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [31]:
cv.best_params_


{'n_estimators': 300, 'max_depth': 5, 'criterion': 'entropy'}